# Projeto - Detector de _tweets_ Antivacina - Coleta
Detector de tweets antivacina | Esse projeto tem como objetivo raspar tweets recentes da API oficial do Twitter, fazer uma análise exploratória das métricas e treinar e classificar automaticamente os tweets como potenciais pró-vacina ou antivacina. 

Projeto realizado como trabalho final da disciplina Pensamento Computacional, do programa de pós-graduação em Jornalismo de Dados, Automação e Data Storytelling do Insper

Profs: Guilherme Dias Felitti e Alvaro Jusen

OBS: Requisições realizadas localmente por problemas de RAM do Colab

# Etapa 1 - Primeiros passos
Importando bibliotecas que serão utilizadas na coleta

In [1]:
# importando bibliotecas para requição e AED
import pandas as pd
import os
import requests
import json

# bibliotecas de suporte para requisição específica da API do Twitter
import csv
import datetime
import time
import dateutil.parser
import unicodedata

In [2]:
# Criando função para realizar autentificação do App
# os.environ['TOKEN'] = '****' # chave de autenficação que o Twitter fornece (não funciona se não tiver uma)
def autentificar():
    return os.getenv('TOKEN')

In [3]:
# Criando função que retornará os headers para fazer a requisição
def criando_header(bearer_token):
    headers = {'Authorization': f'Bearer {bearer_token}'}
    return headers

In [4]:
# criando URL para realizar a requisição / definição dos parâmetros
def criando_URL(palavra_chave, resultado_maximo):
    # endpoint que será utilizada
    endpoint = 'https://api.twitter.com/2/tweets/search/recent'

    # parâmetros que serão utilizados
    parametros = {'query': palavra_chave,
                  'max_results': resultado_maximo,
                  'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                  'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                  'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                  'next_token': {}}
    return (endpoint, parametros)

# Etapa 2 - Requisição de API e criação do banco de dados do projeto


## Criando funções para requisição no Twitter API v2, fazendo requisição na endpoint "```/2/tweets/search/recent```" 

In [5]:
# criando função para se conectar ao endpoint
def conecta_endpoint(url, headers, parametros, next_token = None):
    parametros['next_token'] = next_token
    response  = requests.request('GET', url, headers = headers, params = parametros)
    print(f'Código de resposta do Endpoint: {str(response.status_code)}')
    if response.status_code != 200:
        raise Exception(response.status_code, requests.Response.text)
    return response.json()

In [7]:
# criando função para organizar os dados da coleta dentro de uma lista
def organiza_dados(resposta):

    # Criando variáveis para guardar em dicionário
    data = list()

    # Salvando dados de Data do Twitter
    for valores in range(len(resposta)):
        for l in range(len(resposta[valores]['data'])):
            data.append(resposta[valores]['data'][l])

    # transformando os dados de data em dataframe
    dataframe_data = pd.json_normalize(data)
    dataframe_data = dataframe_data.explode('referenced_tweets', ignore_index=True)
    normal = pd.json_normalize(dataframe_data['referenced_tweets'])
    dataframe_data = dataframe_data.join(normal, how = 'left', lsuffix='left', rsuffix='right').drop(columns = ['referenced_tweets'])

    return dataframe_data


In [ ]:
#Testando a captura
bearer_token = autentificar()
headers = criando_header(bearer_token)
palavra_chave = "vacina"
resultado_maximo = 10
url = criando_URL(palavra_chave, resultado_maximo)
resposta_coleta2 = conecta_endpoint(url[0], headers, url[1])
print(json.dumps(resposta_coleta2, indent = 4, sort_keys = True))

## Realizando requisição usando as funções definidas e armazenando resultado em um dataframe.

In [ ]:
# Realizando a captura automatizada
# Autentificando e criando query da busca
bearer_token = autentificar()
headers = criando_header(bearer_token)
palavra_chave = 'vacina OR vacinação OR antivacina'
resultado_maximo_pesquisa = 100
valor_maximo = 10000

# Criando laço de repetição para capturar até que o resultado máximo seja alcançado
contador = 0
flag = True
next_token = None
coleta = list()
while flag:
    # se o número máximo da coleta for atingido
    if contador >= valor_maximo:
        print('Atingi o valor máximo da coleta. Ufa!')
        break
   
    print(f'Token: {next_token}')
    url = criando_URL(palavra_chave, resultado_maximo_pesquisa)
    resposta_coleta = conecta_endpoint(url[0], headers, url[1], next_token)
    total_resultado_rodada = resposta_coleta['meta']['result_count']

    # se a próxima página existir
    if 'next_token' in resposta_coleta['meta']:
        print(f'Token: {next_token}')
        next_token = resposta_coleta['meta']['next_token']
        if total_resultado_rodada is not None and total_resultado_rodada > 0 and next_token is not None:         
            contador += total_resultado_rodada
            coleta.append(resposta_coleta)
            time.sleep(5)
 
    # se for a última página para coleta
    else:
        if total_resultado_rodada is not None and total_resultado_rodada > 0:
            coleta.append(resposta_coleta)
            contador += total_resultado_rodada
            time.sleep(5)
        
        # desativando o laço infinito
        flag = False
        next_token = None

    time.sleep(5)
dataframe_coleta = organiza_dados(coleta)
print(f'Coleta concluída com sucesso!')

In [ ]:
# Exportando arquivos de coleta para csv
dataframe_coleta.to_csv('dataset2.csv')